# Finding the pencil in the haystack: Identifying rare phenomena with an experiment at the Electron Ion Collider

This project aims to categorize different types of phenomena that could potentially be observed with an experiment at the proposed Electron Ion [Collider](https://en.wikipedia.org/wiki/Collider) (EIC), which will collide electrons with protons at nearly the speed of light. We are searching for a particularly rare phenomenon, the conversion of electrons into <a href="https://en.wikipedia.org/wiki/Tau_(particle)">tau leptons</a>, which would profoundly change our understanding of nature.

The [Standard Model of Elementary Particle Physics](https://en.wikipedia.org/wiki/Standard_Model) is a model to decribe nature. It includes the building blocks of all visible matter in the universe and the forces acting between them. Over the past decades, the Standard Model has had enormous success in precisely describing the data from experiments and making accurate predictions. But, it cannot answer all our questions about nature. Therefore, the search for potential phenomena [beyond the Standard Model](https://en.wikipedia.org/wiki/Physics_beyond_the_Standard_Model) continues with more dedication than ever.

One example of such a 'phenomenon of interest' is the conversion of electrons into tau leptons. So far, no experiment has seen this, and the EIC is uniquely suited to potentially detect this conversion better than any other facility before. But, it is challenging to identify with certainty the rare electron-proton collisions in wich the electron turns into a tau lepton. If they occur, we expect to see about one such conversion per year- compared to an overall rate of thousands of electron-proton collisions per second.


## What do we 'see' when electrons and protons collide?

<br>

<img src="figures/tau_signature.jpg" alt="Experimental signature of a tau lepton decaying into pions." style="width: 400px;"/>

Experiments at high energy particle colliders like the EIC are designed to measure as many as possible of the particles flying away from the point where the collision happens. They measure the direction, energy, and electric charge of these particles (we can neglect the difference between particle energy and momentum because the particles move close to the speed of light). If multiple particles can be grouped together based on the direction they are flying in, they are being considered a <a href="https://en.wikipedia.org/wiki/Jet_(particle_physics)">jet</a>. Different jet finding algorithms exist that apply differnt criteria for this grouping.

Tau leptons decay so fast that we cannot measure them directly with an EIC experiment. However, we __can__ measure their decay products and use that information to identify the original tau lepton. For this study, we focus on tau decays that result in three charged [pions](https://en.wikipedia.org/wiki/Pion) (and a neutral pion and a neutrino, which escapes direct detection). These pions form a characteristically narrow and jet-like cone, which is typically narrower and contains fewer particles than the ubiquitous hadron jets. To identify tau leptons, we need to find an effective way to distinguish tau-jets from hadron jets. This project aims to use a machine learning algorithm to accomlish this.


## The data
For this project, we are considering two possible phenomena that can occur when an electron and a proton collide at very high energies:
1. The electron scatters off the proton, knocks a quark out of the proton, and continues its way. [Deep Inelastic Scattering](https://en.wikipedia.org/wiki/Deep_inelastic_scattering) (DIS)
2. For the simulations used in this study, we assume a model of physics beyond the Standard Model that includes [Leptoquarks](https://en.wikipedia.org/wiki/Leptoquark). Leptoquarks are hypothetical particles that combine properties of quarks and leptons. In addition, they mediate identity changes for charged leptons.

The test and training data include simulations of what a detector at an EIC would measure for each of these two classes of phenomena. Each row in the data files corresponds to a simulated elctron-proton collision. The columns are: ...

Jet candidate characteristics:
1. Energy...
2. Particles...
3. Direction (eta)...

## 1 Preprocessing

Import libraries.

In [1]:
# import libraries
import pandas as pd
import numpy as np

### 1.1 Reading, cleaning, and scaling the data

In [2]:
# read data
data = pd.read_csv('data/LeptoAna_r05_p250_e20.csv')
#data = data.astype('float32')
#data = data.dropna(axis=0)

In [3]:
# replace values: DIS = 0, tau = 1
# note whitespace ' ' before ' DIS' and ' tau'
#map_replace = {
#'jet_type':{
#    ' DIS':0,
#    ' tau':1
#}
#}

#data.replace( map_replace, inplace=True )

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14363 entries, 0 to 14362
Data columns (total 49 columns):
Row                          14363 non-null int64
event                        14363 non-null int64
evtgen_is_tau                14363 non-null int64
evtgen_tau_etotal            6556 non-null float64
evtgen_tau_eta               6556 non-null float64
evtgen_tau_phi               6556 non-null float64
evtgen_tau_decay_prong       14363 non-null int64
evtgen_tau_decay_hcharged    14363 non-null int64
evtgen_tau_decay_lcharged    14363 non-null int64
evtgen_is_uds                14363 non-null int64
evtgen_uds_etotal            6519 non-null float64
evtgen_uds_eta               6519 non-null float64
evtgen_uds_phi               6519 non-null float64
jet_id                       14363 non-null int64
jet_eta                      14363 non-null float64
jet_phi                      14363 non-null float64
jet_etotal                   14363 non-null float64
jet_etrans                   

In [5]:
data.head(1)

,Row,event,evtgen_is_tau,evtgen_tau_etotal,evtgen_tau_eta,evtgen_tau_phi,evtgen_tau_decay_prong,evtgen_tau_decay_hcharged,evtgen_tau_decay_lcharged,evtgen_is_uds,...,jetshape_emcal_econe_r02,jetshape_emcal_econe_r03,jetshape_emcal_econe_r04,jetshape_emcal_econe_r05,tracks_count_r02,tracks_count_r04,tracks_rmax_r02,tracks_rmax_r04,tracks_chargesum_r02,tracks_chargesum_r04
0,2,0,1,34.829021,0.221435,0.760311,3,3,0,0,...,4.608803,4.749965,4.966215,5.408717,1,1,0.073429,0.073429,-1,-1


In [6]:
data['evtgen_is_tau'].value_counts()

0    7807
1    6556
Name: evtgen_is_tau, dtype: int64

## 1.2 Feature selection

In [7]:
#feature_cols = ['n_Above_0p1', 'eta_average', 'Delta_phi_std', 'tower_energy_sum']
#target_col = 'jet_type'
#feature_cols = ['tracks_count_r04', 'tracks_chargesum_r04', 'tracks_rmax_r04', 'jetshape_radius']
feature_cols = [
#    'jet_eta',
#    'jet_phi',
    'jet_etotal',
    'jet_etrans',
    'jet_ptotal',
    'jet_ptrans',
    'jet_minv',
    'jet_mtrans',
    'jet_ncomp',
    'jet_ncomp_above_0p1',
    'jet_ncomp_above_1',
#    'jet_ncomp_above_10',
    'jet_ncomp_emcal',
    'jetshape_radius',
    'jetshape_rms',
    'jetshape_r90',
    'jetshape_econe_r01',
    'jetshape_econe_r02',
    'jetshape_econe_r03',
    'jetshape_econe_r04',
    'jetshape_econe_r05',
    'jetshape_emcal_radius',
    'jetshape_emcal_rms',
    'jetshape_emcal_r90',
    'jetshape_emcal_econe_r01',
    'jetshape_emcal_econe_r02',
    'jetshape_emcal_econe_r03',
    'jetshape_emcal_econe_r04',
    'jetshape_emcal_econe_r05',
#    'tracks_count_r02',
    'tracks_count_r04',
#    'tracks_rmax_r02',
    'tracks_rmax_r04',
#    'tracks_chargesum_r02',
    'tracks_chargesum_r04']

target_col = 'evtgen_is_tau'

features = data[ feature_cols ]
target = data[ target_col ]
target.value_counts()

0    7807
1    6556
Name: evtgen_is_tau, dtype: int64

## 1.3 Splitting training and test data sets

In [8]:
pass

# 2 Training tau-jet classification
## 2.1 Model selection

In [9]:
from sklearn.model_selection import train_test_split

# create training and testing vars
#X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.8)
#print (X_train.shape, y_train.shape)
#print (X_test.shape, y_test.shape)

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_predict, KFold

penalty = {
    0: 100,
    1: 1
}

#lr = LogisticRegression(class_weight=penalty)

#lr = DecisionTreeClassifier(class_weight=penalty, max_depth=30)

lr = AdaBoostClassifier(random_state=1)

#lr = RandomForestClassifier(class_weight=penalty, random_state=1, max_depth=20)
#lr = RandomForestClassifier(class_weight='balanced', random_state=1)
#lr = RandomForestClassifier(random_state=1)
kf = KFold(features.shape[0], shuffle=True, random_state=1)

predictions = cross_val_predict(lr, features, target, cv=kf)
#predictions = cross_val_predict(lr, features, target, cv=10)

#lr.fit( features, target )
#predictions = lr.predict(features)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (target == 0)
fp = len(data[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (target == 1)
tp = len(data[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (target == 1)
fn = len(data[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (target == 0)
tn = len(data[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print( "True positive: "+str(tp))
print( "True negativee: "+str(tn))
print( "False positive: "+str(fp))
print( "False negative: "+str(fn))
print( "True Positive Rate: "+str(tpr) )
print( "False Positive Rate: "+str(fpr) )

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


True positive: 6020
True negativee: 7086
False positive: 721
False negative: 536
True Positive Rate: 0.9182428309945089
False Positive Rate: 0.09235301652363263


## 2.2 Cross-validation

In [11]:
pass

## 2.3 Performance metrics


In [12]:
pass

##  2.4 Hyperparameter optimization

In [13]:
pass

# 3 Evaluation

In [14]:
pass

# Conclusion

Lorem ipsum...